# Zufriedenheit

## Importe

In [49]:
import pandas as pd
import plotly.io as pio
import plotly.express as px

## Template

In [50]:
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="Title",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

### load the data summed up

In [51]:
years = [str(year) for year in range(2013, 2025)]

df = pd.read_excel("data.xlsx", sheet_name=years)

data_list = []

for year, df in df.items():
    df['Jahr'] = year

    data_list.append(df)

final_df = pd.concat(data_list, ignore_index=True)



### clean the data

In [52]:
df = final_df[final_df.columns[~final_df.columns.str.contains(r'\*')]] # drop * in the data
df.columns = df.columns.str.replace('*', '', regex=False)# just to be sure
df = df.rename(columns={'Master Getränketechnologi':'Master Getränketechnologie'})# fix a typo
clean_columns = {}
for col in df.columns:
    clean_name = col.strip()  # get rid of other human typos
    if clean_name in clean_columns:
        clean_columns[clean_name].append(col)
    else:
        clean_columns[clean_name] = [col]

# smack together the similar col names 
merged_df = df[['Jahr']].copy()
for clean_name, original_cols in clean_columns.items():
    if len(original_cols) > 1:
        merged_df[clean_name] = df[original_cols].sum(axis=1)
    else:
        merged_df[clean_name] = df[original_cols[0]]
df = merged_df.fillna(value=0) # use if needed for plotting

### transform

In [53]:
df = df[df['Variable'].isin(['Zufriedenheit'])]
df = df[df['Variable'] == 'Zufriedenheit'].drop(columns='Variable')
df = df[df["Category"] != "Gesamt (%)"]
df = df.groupby(['Jahr', 'Category']).sum().reset_index()


In [54]:
anzahl_row = df[df['Category'] == 'Anzahl']
for year in df['Jahr'].unique():
    gesamtzahlen = anzahl_row[anzahl_row['Jahr'] == year].drop(columns=['Jahr', 'Category']).values.flatten()

    for col in df.columns[2:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Umwandlung in numerisch, fehlerhafte Werte werden zu NaN

    for idx, row in df[df['Jahr'] == year].iterrows():
        if row['Category'] != 'Anzahl':
            relative_prozent = row.drop(['Jahr', 'Category']).values
            absolute_werte = (relative_prozent / 100) * gesamtzahlen
            df.loc[idx, df.columns[2:]] = absolute_werte
            df.round(0)
df = df[df['Category'] != 'Anzahl']
df['sum']= df.sum(axis=1, numeric_only=True).round(0)
df = df.get(key=['Jahr', 'Category', 'sum'])

In [55]:
category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]
df['Category'] = pd.Categorical(df['Category'], categories=category_order, ordered=True)

In [56]:
df_rel = df.pivot(index="Jahr", columns="Category")
df_rel.columns = df_rel.columns.droplevel()
df_rel = df_rel.apply(pd.to_numeric, errors='coerce')


In [57]:
df_rel = (
    df_rel.div(df_rel.sum(axis=1), axis=0).mul(100)
)

### plot

In [58]:
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="Relative Verteilung der Antworten zu Zufriedenheit in verschiedenen Kategorien von 2013 bis 2024",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)

pio.templates["infoviz"] = infoviz_template

category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]

df_melted = df_rel.reset_index().melt(id_vars="Jahr", var_name="Category", value_name="sum")
df_melted.rename(columns={"index": "Jahr"}, inplace=True)

df_melted['Category'] = pd.Categorical(df_melted['Category'], categories=category_order, ordered=True)

df_melted = df_melted.sort_values(by=['Jahr', 'Category'])

fig = px.bar(df_melted, x="Jahr", 
             y="sum", 
             color="Category", 
             title="Entwicklung der Zufriedenheit im Fachbereich 09", 
             color_discrete_sequence=px.colors.qualitative.Safe, 
             template="infoviz",
             barmode="stack",
             labels={
                 'Jahr': 'Jahr', 
                 'sum': 'Relativer Anteil der Antworten',
                 'Category': 'Kategorien'
             })


fig.update_layout(
    width=1400,
    height=500,
)
fig.show()
fig.write_image("Plots/pdf/zufriedenheit1.pdf")


In [59]:
category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]

df_melted = df_rel.reset_index().melt(id_vars="Jahr", var_name="Category", value_name="sum")
df_melted.rename(columns={"index": "Jahr"}, inplace=True)

df_melted['Category'] = pd.Categorical(df_melted['Category'], categories=category_order, ordered=True)

df_melted = df_melted.sort_values(by=['Jahr', 'Category'])

fig = px.bar(df_melted, x="Jahr", 
             y="sum", 
             color="Category", 
             title="Zufriedenheit der Studierenden im Fachbereich 09", 
             subtitle="",
             color_discrete_sequence=px.colors.qualitative.Safe, 
             template="infoviz",  
             barmode="group",  
             labels={
                 'Jahr': 'Jahr', 
                 'sum': 'Relativer Anteil der Antworten',
                 'Category': 'Kategorien'
             })
fig.update_layout(
    width=1400,
    height=700,
)
fig.show()
fig.write_image("Plots/pdf/zufriedenheit2.pdf")

In [60]:
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="Absolute Verteilung der Antworten zu Zufriedenheit in verschiedenen Kategorien von 2013 bis 2024",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)

pio.templates["infoviz"] = infoviz_template

category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]

df['Category'] = pd.Categorical(df['Category'], categories=category_order, ordered=True)

df = df.sort_values(by=['Jahr', 'Category'])

fig = px.bar(df, x="Jahr", 
             y="sum", 
             color="Category", 
             title="Zufriedenheit mit dem Studium im Fachbereich", 
             color_discrete_sequence=px.colors.qualitative.Safe, 
             template="infoviz",
             
             labels={
                 'Jahr': 'Jahr', 
                 'sum': 'Anzahl der Studierenden',
                 'Category': 'Kategorien'
             })

fig.update_traces(
    name="gar nicht zufrieden", 
    selector=dict(name='gar nicht zufrieden (%)') 
)
fig.update_traces(
    name="weniger zufrieden",
    selector=dict(name='weniger zufrieden (%)')
)
fig.update_traces(
    name="mittelmäßig zufrieden",
    selector=dict(name='mittelmäßig zufrieden (%)')
)
fig.update_traces(
    name="eher zufrieden",
    selector=dict(name='eher zufrieden (%)')
)
fig.update_traces(
    name="sehr zufrieden",
    selector=dict(name='sehr zufrieden (%)')
)

fig.update_layout(
    width=1400,
    height=500,
)
fig.show()
fig.write_image("Plots/pdf/zufriedenheit3.pdf")

In [61]:


category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]

df['Category'] = pd.Categorical(df['Category'], categories=category_order, ordered=True)

df = df.sort_values(by=['Jahr', 'Category'])

fig = px.bar(df, x="Jahr", 
             y="sum", 
             color="Category", 
             title="Zufriedenheit mit dem Studium im Fachbereich", 
             subtitle="",
             color_discrete_sequence=px.colors.qualitative.Safe, 
             template="infoviz",  
             barmode="group", 
             labels={
                 'Jahr': 'Jahr', 
                 'sum': 'Anzahl der Studierenden',
                 'Category': 'Kategorien'
             })

fig.update_layout(height=700)

fig.update_traces(
    name="gar nicht zufrieden", 
    selector=dict(name='gar nicht zufrieden (%)') 
)
fig.update_traces(
    name="weniger zufrieden",
    selector=dict(name='weniger zufrieden (%)')
)
fig.update_traces(
    name="mittelmäßig zufrieden",
    selector=dict(name='mittelmäßig zufrieden (%)')
)
fig.update_traces(
    name="eher zufrieden",
    selector=dict(name='eher zufrieden (%)')
)
fig.update_traces(
    name="sehr zufrieden",
    selector=dict(name='sehr zufrieden (%)')
)

fig.update_layout(
    width=1400,
    height=700,
)
fig.show()
fig.write_image("Plots/pdf/zufriedenheit4.pdf")